In [1]:
#run data collection before this
import numpy as np
import cv2

In [2]:
with_mask=np.load('with_mask.npy')
without_mask=np.load('without_mask.npy')

In [3]:
with_mask.shape

(200, 50, 50, 3)

In [4]:
without_mask.shape

(200, 50, 50, 3)

In [5]:
with_mask=with_mask.reshape(200,50*50*3)
without_mask=without_mask.reshape(200,50*50*3)

In [6]:
with_mask.shape

(200, 7500)

In [7]:
without_mask.shape

(200, 7500)

In [8]:
X=np.r_[with_mask,without_mask]

In [9]:
X.shape

(400, 7500)

In [10]:
labels = np.zeros(X.shape[0])

In [11]:
labels[200:]=1.0

In [12]:
names={0 :'Mask',1: 'No mask'}

In [13]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train,x_test,y_train,y_test=train_test_split(X,labels,test_size=0.25)

In [16]:
x_train.shape

(300, 7500)

In [17]:
from sklearn.decomposition import PCA

In [18]:
pca=PCA(n_components=3)
x_train=pca.fit_transform(x_train)

In [19]:
x_train[0]

array([1178.29212721, 1280.97756927, -734.54621038])

In [20]:
x_train.shape

(300, 3)

In [21]:
x_train,x_test,y_train,y_test=train_test_split(X,labels,test_size=0.10)

In [22]:
x_test.shape

(40, 7500)

In [23]:
svm=SVC()
svm.fit(x_train,y_train)

SVC()

In [24]:
#x_test=pca.transform(x_test)
y_pred=svm.predict(x_test)

In [25]:
accuracy_score(y_test,y_pred)

1.0

In [26]:
def check_mask():
    path_to_haar_data = r'E:\Me\UPES\Hackathon\CSA 1.0\Face detector\face_data.xml'
    haar_data= cv2.CascadeClassifier(r''.join(path_to_haar_data))
    capture=cv2.VideoCapture(0)
    data=[]
    while True:
        flag,img=capture.read()
        if flag:
            faces=haar_data.detectMultiScale(img)
            for x,y,w,h,in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),3)
                face=img[y:y+h,x:x+w,:]
                face=cv2.resize(face,(50,50))
                face=face.reshape(1,-1)
               # face=pca.transform(face)
                pred=svm.predict(face)
                n=names[int(pred)]
                print(n)
                data.append(n)
            #cv2.imshow('result',img)
            #if cv2.waitKey(2)==27: 
            if len(data) > 5:
                break
    capture.release()
    cv2.destroyAllWindows()
    count = 0
    for result in data:
        if result == "Mask":
            count += 1
    if (count < 3):
        return(30) 
    else:
        return(1800)

In [ ]:
def notify_mask(t):
    print(t)
    if t == -1:
        return(print("Camera not detected, contact support."))
    import time
    from plyer import notification
    time.sleep(t)
    if t == 30:
        notification.notify(title = "ALERT", message = "Please wear a mask", app_icon = r"E:\Me\UPES\Hackathon\CSA 1.0\Notification\icon.ico")
        return
    else:
        notification.notify(title = "ALERT", message = "Please confirm that you are wearing a mask.", app_icon = r"E:\Me\UPES\Hackathon\CSA 1.0\Notification\icon.ico")
        time.sleep(10)
        return
c = 0
while c < 5:
    notify_mask(check_mask())
    c += 1

No mask
No mask
No mask
No mask
No mask
No mask
30
No mask
No mask
No mask
No mask
No mask
No mask
30
